In [ ]:
import pandas as pd

In [ ]:
import numpy as np

# The Data Sets:
We used New York city public schools data set from the following link: https://data.cityofnewyork.us/browse?
Here are the list to all of the datasets I’ll be using:

SAT scores by school — SAT scores for each high school in New York City.
School attendance — attendance information on every school in NYC.
Math test results — math test results for every school in NYC.
Class size — class size information for each school in NYC.
AP test results — Advanced Placement exam results for each high school. Passing
AP exams can get you college credit in the US.
Graduation outcomes — percentage of students who graduated, and other outcome information.
Demographics — demographic information for each school.
School survey — surveys of parents, teachers, and students at each school.
School district maps — contains information on the layout of the school districts, so that we can map them out.

In [ ]:
AllDataSets = ["Graduation_Outcomes.csv","Math_Test_Results.csv", "demographics.csv",  "College_Board__School.csv", "Class_Size.csv", "sat_results.csv"]

I will Loop through each data file we will be working on.
Read the file into a Pandas DataFrame.
Put each DataFrame into a Python dictionary.

In [ ]:
data = {}

In [ ]:
import os

In [ ]:
print("pwd=" + os.getcwd())

In [ ]:
for i in AllDataSets:
    d=pd.read_csv("NewYorkCityPublickSchoolData/{0}".format(i))
    data[i.replace(".csv", "")] = d


# Understanding the data
Once I’ve read the data in, Its time to use the head method on DataFrames to print the first 5 lines of each DataFrame:

In [ ]:
for i,j in data.items():
    print("\n" + i +"\n" )
    print(j.head())
    

We can start to see some useful patterns in the datasets:

Most of the datasets contain a DBN column
Some of the datasets appear to contain multiple rows for each school (repeated DBN values), which means we’ll have to do some preprocessing.

In [ ]:
data["Math_Test_Results"].head()

In [ ]:
data["Math_Test_Results"]["DBN"].head

In [ ]:
data["Class_Size"].head()

In [ ]:
data["Class_Size"].head

Constructing Class size DBN, as required to link all the data sets using the mutual column "DBN".

In [ ]:
data["Class_Size"]["DBN"]=data["Class_Size"].apply(lambda x: "{0:2d}{1}".format(x["CSD"],x["SCHOOL CODE"]), axis=1)

In [ ]:
data["Class_Size"]["DBN"].head

# Its time to add the surveys:
we’ll add the survey data into our data dictionary, and then combine all the datasets afterwards. The survey data consists of 2 files, one for all schools, and one for school district 75. We’ll need to write some code to combine them. In the below code, we’ll:

Read in the surveys for all schools using the windows-1252 file encoding.
Read in the surveys for district 75 schools using the windows-1252 file encoding.
Add a flag that indicates which school district each dataset is for.
Combine the datasets into one using the concat method on DataFrames.

In [ ]:
s1=pd.read_csv("NewYorkCityPublickSchoolData/survey1_gened.txt", delimiter="\t", encoding='windows-1252')

In [ ]:
s2=pd.read_csv("NewYorkCityPublickSchoolData/survey1_d75.txt", delimiter="\t", encoding='windows-1252')

In [ ]:
s1["d75"]=False

In [ ]:
s2["d75"]=True

In [ ]:
survey=pd.concat([s1,s2], axis=0)

In [ ]:
survey.head()

In [ ]:
survey["DBN"]=survey["dbn"]

In [ ]:
survey_fields = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]

In [ ]:
survey=survey.loc[:,survey_fields]

In [ ]:
data["survey"]=survey

In [ ]:
survey.shape

In [ ]:
survey.head()

In [ ]:
data["Class_Size"].head()

In [ ]:
class_size=data["Class_Size"]

In [ ]:
class_size=class_size[class_size["GRADE "]=="09-12"]

In [ ]:
class_size=class_size[class_size["PROGRAM TYPE"]== "GEN ED"]

In [ ]:
class_size=class_size.groupby("DBN").agg(np.mean)

In [ ]:
class_size.reset_index(inplace=True)

In [ ]:
data["Class_Size"]=class_size

In [ ]:
data["Class_Size"].head()

In [ ]:
demo=data["demographics"]

In [ ]:
demo=demo[demo["schoolyear"]==20112012]

In [ ]:
demo.head()

# Data Condensation

In [ ]:
data["demographicsc"]=demo

In [ ]:
data["Math_test_Results"] = data["Math_Test_Results"][data["Math_Test_Results"]["Year"]==2011]

In [ ]:
data["Math_test_Results"] = data["Math_test_Results"][data["Math_test_Results"]["Grade"] == '8']

In [ ]:
data["Math_test_Results"].head()

In [ ]:
data["Graduation_Outcomes"]= data["Graduation_Outcomes"][data["Graduation_Outcomes"]["Cohort"]=="2006"]

In [ ]:
data["Graduation_Outcomes"] = data["Graduation_Outcomes"][data["Graduation_Outcomes"]["Demographic"] == "Total Cohort"]

In [ ]:
data["Graduation_Outcomes"].head()

# Computing Key Variables

In [ ]:
cols = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']

In [ ]:
for c in cols:
    data["sat_results"][c] = data["sat_results"][c].convert_objects(convert_numeric=True)
    data['sat_results']['sat_score'] = data['sat_results'][cols[0]] + data['sat_results'][cols[1]] + data['sat_results'][cols[2]]

In [ ]:
data["school_Attendance"]['lat'] = data["school_Attendance"]['Location 1'].apply(lambda x: x.split("\n")[-1].replace("(", "").replace(")", "").split(", ")[0])
data["school_Attendance"]['lon'] = data["school_Attendance"]['Location 1'].apply(lambda x: x.split("\n")[-1].replace("(", "").replace(")", "").split(", ")[1])
for c in ['lat', 'lon']:
    data["school_Attendance"][c] = data["school_Attendance"][c].convert_objects(convert_numeric=True)

In [ ]:
for i,j in data.items():
    print(i)
    print(j.head())

In [ ]:
data["Math_Test_Results"]["DBN"].head()

In [130]:
flat_data_names = [k for k,v in data.items()]
flat_data = [data[k] for k in flat_data_names]
full = flat_data[0]
for i, f in enumerate(flat_data[1:]):
    name = flat_data_names[i+1]
    print(name)
    print(len(f["DBN"]) - len(f["DBN"].unique()))
    join_type = "inner"
    if name in ["sat_results", "College_Board_School", "Graduation_Outcomes"]:
        join_type = "outer"
    if name not in ["math_test_results"]:
        full = full.merge(f, on="DBN", how=join_type)
    if name not in ["math_test_results"]:
        full = full.merge(f, on="DBN", how=join_type)
full.shape

Math_Test_Results
27346
demographics
8481
College_Board__School
1
Class_Size
0
sat_results
0
survey
0
demographicsc
0
Math_test_Results
0


(651398, 303)

In [131]:
cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']
for col in cols:
full[col] = full[col].convert_objects(convert_numeric=True)
full[cols] = full[cols].fillna(value=0)

IndentationError: expected an indented block (<ipython-input-131-73f9abb57976>, line 3)

In [ ]:
full["school_dist"] = full["DBN"].apply(lambda x: x[:2])

In [ ]:
full = full.fillna(full.mean())

In [ ]:
full.head()

In [ ]:
full.corr()['sat_score']

This gives us quite a few insights that we’ll need to explore:

Total enrollment correlates strongly with sat_score, which is surprising, because you’d think smaller schools, which focused more on the student, would have higher scores.
The percentage of females at a school (female_per) correlates positively with SAT score, whereas the percentage of males (male_per) correlates negatively.
None of the survey responses correlate highly with SAT scores.
There is a significant racial inequality in SAT scores (white_per, asian_per, black_per, hispanic_per).
ell_percent correlates strongly negatively with SAT scores.
Each of these items is a potential angle to explore and tell a story about using the data.

# Data Visualization

In [ ]:
full.plot.scatter(x='total_enrollment', y='sat_score')

In [ ]:
full.plot.scatter(x='ell_percent', y='sat_score')

In [ ]:
show_district_map("ell_percent")

In [ ]:
full.corr()["sat_results"][["rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_tot_11", "com_tot_11", "aca_tot_11", "eng_tot_11"]].plot.bar()

# Gender Impact:


In [ ]:
full.corr()["sat_results"][["male_per", "female_per"]].plot.bar()

AP Scores:
So far, we’ve looked at demographic angles. One angle that we have the data to look at is the relationship between more students taking Advanced Placement exams and higher SAT scores. It makes sense that they would be correlated, since students who are high academic achievers tend to do better on the SAT.

In [ ]:
full["ap_avg"] = full["AP Test Takers "] / full["total_enrollment"]
full.plot.scatter(x='ap_avg', y='sat_score')

In [ ]:
full[(full["ap_avg"] > .3) & (full["sat_score"] > 1700)]["School Name"]